# 參考文獻
https://people.math.ethz.ch/~hjfurrer/teaching/LongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf

# 作法
- (1) 先對股價(S)產生一個全為0的 (n+1) x I 二維矩陣，以及產生一個服從標準常態分配的 (n+1) x I 二維亂數矩陣
- (2) 將產生的亂數代入股價公式，可以計算出 (n+1) x I 的隨機股價矩陣，用以模擬出 n+1 期 I 個路徑的股價  ( n+1是因為第一列為S0 )
- (3) 使用最小平方法跑迴歸 :
    - 迴歸模型 : Y = b0 + b1X + b2(X**2) + u
    - Y : 將第t+1期的預期現金流折現至第t期之現金流
    - X : 第t期的股價
    - u : 隨機誤差
    - t : t = n, n-1,..., 1
    - 有效成對樣本(X,Y)數 n : I個路徑中，在t+1期時預期現金流大於0之個數，亦即在t+1期落在價內的個數 ( 在t+1期處於價外者不列入樣本內 )
    - 因為我們是要評估美式選擇權是否有可能(值得)提前履約，故分析重點只有價內選擇權 ( 選擇權在價內才有可能履約 )
- (4) 以(3)之迴歸模型估計在第t期仍繼續持有選擇權不提前履約的期望價值(Y_hat)
- (5) 將(4)的結果和提前履約所能得到的價值做比較
    - 現在為第t期，進行決策分析
    - 若繼續持有選擇權到t+1期的期望價值較大，則在第t期時不提前履約，則此時第t期現金流=0，第t+1期現金流=第t+1期的預期現金流
    - 若繼續持有選擇權到t+1期的期望價值較小，則在第t期時提前履約，此時第t期現金流=提前履約價值，第t+1期現金流=0 (因為第t期就提前履約)
- (6) 重複(3)~(5)之過程直到t=1為止
- (7) 在t=1時，會得出各個路徑，最早會在哪個時點提前履約
- (8) 將所有路徑能得到的現金流折現到t=0並取平均(除以I)，則此「平均可得現金流」即為用最小平方蒙地卡羅模擬出來的美式選擇權價格

# 導入套件

In [1]:
import math
import random
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline

# 參數設定區域
- S0 : 現在股價 ( 固定外生變數 )
- St : t時點股價 ( 隨機變數，t的函數)
- K : 履約價 ( 假設為固定外生變數，不隨時間t改變 )
- r : 無風險利率 ( 假設為固定外生變數，不隨時間T改變 )
- sigma : 股票報酬率標準差 ( 波動度，為固定外生變數，假設不隨時間T改變 )
- T : 到期日 (從0時點起算。單位:年)
- n : 期數
- t : 某一時點 (0~T之間任意時點，可能數值有無限多個。單位:年)
- dt : 每一期長度
- z : 標準常態分配亂數矩陣
- S : 隨機股價矩陣

In [2]:
S0 = 100         # 現在股價 ( 固定外生變數 )
K = 110          # 履約價
r = 0.05         # 無風險利率
sigma = 0.25     # 股票波動度 ( 股票報酬率標準差 )
T = 1.0          # 到期日 ( 單位:年 )
n = 50           # 期數
I = 50000        # 模擬次數
dt = T / n       # 每一期長度

option = 'put'   # 輸入 call 或 put 

# Step 1 建立 n x I 的隨機股價矩陣

In [3]:
def random_path_stock(S0, r, sigma, T, I, n) :
    S = np.zeros( (n+1, I) )                     # 建立內容全為0的股價矩陣
    z = np.random.standard_normal( (n+1, I) )    # 建立服從標準常態分配的亂數矩陣
                                                 # 兩個皆用n+1的目的為 t=0對應矩陣第0列，t=1對應矩陣第1列,...,t=50對應矩陣第50列
        
    S[0] = S0     # 不論在哪個路徑，t=0時股價都等於S0
    
    for t in range(1,n+1) :    # 用於產生 t=1~t=50 的隨機股價 ( 每個時點t皆會產生I個股價 )
        S[t] = S[t-1]*np.exp( ( r - sigma**2 / 2 )*dt + sigma*math.sqrt(t)*z[t] )  # t時點的股價( S[t] )
                                                                                   # 是上一期股價( S[t-1] )和
                                                                                   # 本期隨機亂數( z[t] )的函數
                                                                                   # 並且兩個時點(一期)的間隔為dt
    return S     

In [4]:
S = random_path_stock(S0, r, sigma, T, I, n)

# Step 2 計算出選擇權提前履約之隨機收益矩陣

In [5]:
if option == 'call' :              
    payoff = np.maximum(S - K, 0)    # call履約價值
else :
    payoff = np.maximum(K - S, 0)    # put履約價值

In [6]:
payoff2 = np.copy( payoff )   # 因為後面會更改payoff內的元素，並且更改前後的payoff都會用到，
                              # 故使用np.copy，使更改前後的payoff獨立，更改後不會影響到原來的payoff
                              # 注意，因為後面運算中，payoff裡面沒有「子物件」，不需要用到 np.deepcopy

# Step 3 建立迴歸方程式
- 使用最小平方法跑迴歸 
- 迴歸模型 : Y = b0 + b1X + b2(X**2) + u
    - Y : 將第t+1期的預期現金流折現至第t期之現金流
    - X : 第t期的股價
    - u : 隨機誤差
    - t : t = n, n-1, ..., 1
    - 有效成對樣本(X,Y)數 n : I個路徑中，在t+1期時預期現金流大於0之個數，亦即在t+1期落在價內的個數 ( 在t+1期處於價外者不列入樣本內 )
    - 因為我們是要評估美式選擇權是否有可能(值得)提前履約，故分析重點只有價內選擇權 ( 選擇權在價內才有可能履約 )
- 在第t期決定繼續持有選擇權之價值 : hold_value_t

In [7]:
discount_rate = np.exp(-r*dt)    # 折現因子

- 影片作法跟論文原始做法之不同處
    - (1) 影片作法為不論第i個路徑在第t+1期是否落在價內皆列入迴歸樣本，論文做法只有價內路徑列入迴歸樣本
- 注意， 論文「繼續持有選擇權到t+1期的期望價值較大，則在第t期時不提前履約，則此時第t期現金流=0，第t+1期現金流=第t+1期的預期現金流，並且t-1期之Y為t+1期現金流折現至t-1」
- 而此處寫法是 : 既然第t期值為0，且第t-1期現金流為t+1期折現至t-1，那麼將第t期現金流改為第t+1期現金流折現，再將此第t期現金流折現至第t-1期，結果與論文相同，但程式較好寫」

In [8]:
for t in range(n-1, 0, -1) :   # 共跑 51-2 次迴歸 (矩陣共有51列，t=0不跑迴歸，t=50~t=1每兩期跑一次迴歸，共可跑49次)
    reg = np.polyfit( S[t], payoff2[t+1]*discount_rate, deg = 2 ) 
    hold_value_t = np.polyval(reg, S[t])    # 將 X 代入每次估計出來的樣本迴歸式而得之「Y_hat」存入hold_value_t中
    payoff2[t] = np.where( hold_value_t[t] > payoff[t], payoff2[t+1]*discount_rate, payoff[t] )
    # 現在為第t期，進行決策分析
    # (1)若繼續持有選擇權到t+1期的期望價值較大，則在第t期時不提前履約，則此時第t期現金流=第t+1期現金流折現至第t期
    # (2)若繼續持有選擇權到t+1期的期望價值較小，則在第t期時提前履約，此時第t期現金流=提前履約價值，
    #                                                           第t+1期現金流=0(因為第t期就提前履約)
    # 注意，跑迴歸時，一律使用「更改後的payoff2」
    #      若發生(1)時，會使用「更改後的payoff2」進行計算
    #      若發生(2)時，會使用「更改前的payoff」進行計算
option_value = np.mean( payoff2[1]*discount_rate )

In [9]:
option_value

73.16396245628515

先按照影片內容做，論文原始作法再詢問

In [10]:
# for t in range(n, 0, -1)   # 共跑 51-1 次迴歸 ( 矩陣共有51列，t=0~t=50 )
#     for i in range(I+1) :  # 共有I個路徑
#         if payoff[t+1,i] > 0 :     # 對payoff每個元素，若其在t+1期收益大於0(落在價內)才對列入迴歸的樣本中
#             reg = np.polyfit( S[t], payoff[t+1]*discount_rate, deg = 2 )   
#             hold_value_t = np.ployval(reg, S[t])    # 將 X 代入每次估計出來的樣本迴歸式而得之「Y_hat」存入hold_value_t中    